# User Input for a Prometheus Run
=================================

Please run each code block and follow the instructions.
The end result will be a downloadable .csv file with the necessary values.
Ensure that you have python and wget installed on your machine before starting.

There is an accompanying how-to in the "Installations" tab.

In [1]:
# Necessary imports 
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import ipywidgets as widgets
import requests
import xarray as xr

from ipyleaflet import Map, Marker, GeoJSON
from ipywidgets import Output
from IPython.display import display
from datetime import datetime

from etl_prometheus_data import interpolate_grib2_to_point
from file_funcs import set_filenames


In [2]:
# run the etl_nfdf_polygons.py script to get the NFDF polygons
%run etl_nfdb_polygons.py

Shapefile already exists. Downloading the latest data...


In [2]:
# Load the GeoJSON file with geopandas
gdf = gpd.read_file('Canada_perimeters.geojson')

# Ensure CRS is WGS84
gdf = gdf.to_crs(epsg=4326)

# Optionally drop columns you don't need
gdf = gdf.drop(columns=['FIRSTDATE', 'LASTDATE', 'CONSIS_ID'])

# Convert to GeoJSON dict
geojson_data = json.loads(gdf.to_json())

# Center the map on the centroid of all polygons
center = gdf.geometry.unary_union.centroid.coords[0][::-1]
m = Map(center=center, zoom=4)

# Add the GeoJSON layer
geojson_layer = GeoJSON(
    data=geojson_data,
    name='NFDB Polygons',
    style={
        'color': 'red',
        'weight': 1,
        'fillColor': 'orange',
        'fillOpacity': 0.6
    }
)
m.add_layer(geojson_layer)

# add marker that will be updated on click
marker = Marker(location=center)    
m.add_layer(marker)

out = Output()

def handle_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        latlon = kwargs.get('coordinates')
        marker.location = latlon
        with out:
            print(f"Clicked location: {latlon}")

m.on_interaction(handle_map_click)

print("Click on the map to get coordinates")
print("Double-click to zoom in or use the zoom controls")
print("Your final click is the selected location")
display(m, out)

C:\Users\lbuchart\AppData\Local\Temp\ipykernel_6252\2273810375.py:14: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gdf.geometry.unary_union.centroid.coords[0][::-1]


Click on the map to get coordinates
Double-click to zoom in or use the zoom controls
Your final click is the selected location


Map(center=[56.43705508967247, -106.76423227937384], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Output()

In [3]:
coords = marker.location
print(f"The final coordinates you have selected are: {coords}")

# save the map to an HTML file
m.save('nfdb_polygons_map.html')


The final coordinates you have selected are: [47.849294596884526, -53.187844080716324]


In [4]:
# now select model and dates to download ECCC data
from IPython.display import display
model_dropdown = widgets.Dropdown(
    options=['rdps', 'hrdps'],
    value='rdps',
    description='Model:',
)

model_run = widgets.Dropdown(
    options=['00', '06', '12', '18'],
    value='00',
    description='Model Initialization Time:',
)

In [5]:
current_year = datetime.now().year
year_slider = widgets.IntSlider(
    value=int(current_year),
    min=2020,
    max = int(current_year),
    step=1,
    description='Year:',
    continuous_update=False
)

month_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=12,
    step=1,
    description='Month:',
    continuous_update=False
)

day_slider = widgets.IntSlider(
    value=1,
    min=1, 
    max=31,
    step=1,
    description='Day:',
    continuous_update=False
)

forecast_slider = widgets.IntSlider(
    value=48,
    min=24,
    max=72,
    step=6,
    description='Forecast Length (hours):',
    continuous_update=False
)

# Display the widgets
display(model_dropdown, model_run, year_slider, month_slider, day_slider, forecast_slider)

print("Once you are happy with your selections")
print("Move to the next cell to begin the download process")

print("Note that the MSC Datamart only has data from the previous 30 days")

Dropdown(description='Model:', options=('rdps', 'hrdps'), value='rdps')

Dropdown(description='Model Initialization Time:', options=('00', '06', '12', '18'), value='00')

IntSlider(value=2025, continuous_update=False, description='Year:', max=2025, min=2020)

IntSlider(value=1, continuous_update=False, description='Month:', max=12, min=1)

IntSlider(value=1, continuous_update=False, description='Day:', max=31, min=1)

IntSlider(value=48, continuous_update=False, description='Forecast Length (hours):', max=72, min=24, step=6)

Once you are happy with your selections
Move to the next cell to begin the download process
Note that the MSC Datamart only has data from the previous 30 days


In [6]:
model_files = set_filenames(model_dropdown.value, model_run.value, year_slider.value, month_slider.value, day_slider.value, forecast_slider.value)
print(model_files)

Selected Model: rdps
Selected Model Run: 12
Selected Date: 2025-08-10
Forecast Length: 24 hours
                                            full_path  \
0   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
1   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
2   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
3   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
4   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
..                                                ...   
91  https://dd.weather.gc.ca/20250810/WXO-DD/model...   
92  https://dd.weather.gc.ca/20250810/WXO-DD/model...   
93  https://dd.weather.gc.ca/20250810/WXO-DD/model...   
94  https://dd.weather.gc.ca/20250810/WXO-DD/model...   
95  https://dd.weather.gc.ca/20250810/WXO-DD/model...   

                                            extension  \
0   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
1   https://dd.weather.gc.ca/20250810/WXO-DD/model...   
2   https://dd.weather.gc.ca/20250810/WXO-DD/mod

In [ ]:
# download the files using the requests library
import requests

# loop through the model_files DataFrame and download each file
for index, row in model_files.iterrows():
    output_file = row['file']
    file_url = row['full_path']
    try:
        # Send a GET request to download the file
        response = requests.get(file_url, stream=True)
        response.raise_for_status()  # Check for HTTP request errors

        # Write the content to a local file
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"File downloaded successfully: {output_file}")

        # interpolate then delete the data to keep the temp directory clean
        

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")   